In [2]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option("display.max_columns", None)

In [ ]:
display(df.sort_values(by='dias_entrega',ascending=False))

In [ ]:
len(df[(df['dias_entrega'] > 40)])

In [ ]:
print(len(df[(df['customer_state'] != df['seller_state']) & (df['dias_entrega'] > 16)]))

# 4 - Análise e exploraçáo dos dados

In [8]:
# leitura da base

path_df_final_csv = 'd:/tcc_puc_minas/dataframe_final/df_final.csv'

df = pd.read_csv(path_df_final_csv,sep='|')

df['review_score'] = df['review_score'].astype(str)

## 4.1. Análise univariada

In [9]:
var_num = []
var_cat = []

for col in df.columns:
    if df[col].dtypes == 'object':
        var_cat.append(col)
    else:
        var_num.append(col)

print('var_num: ',var_num)
print('var_cat: ',var_cat)

var_num:  ['pib_mun_customer', 'pib_mun_seller', 'product_photos_qty', 'dias_entrega', 'price', 'freight_value']
var_cat:  ['customer_city', 'customer_state', 'atividade_mun_customer', 'seller_city', 'seller_state', 'atividade_mun_seller', 'product_category_name', 'payment_type', 'review_score']


### 4.1.1. Variáveis Quantitativas

#### 4.1 - vizualizando a varável alvo, review_scores

In [ ]:
# criando um dataframe que faz a contagem dos elementos de review_score

review_score_count_series = df['review_score'].value_counts()
print(review_score_count_series)

In [ ]:
df_review_score_count = pd.DataFrame(review_score_count_series)
df_review_score_count = df_review_score_count.reset_index()
df_review_score_count.columns = ['review_score','qtd']
display(df_review_score_count)

In [ ]:
# fazendo um bar plot de review_scores

plt.figure(figsize=(15,7.5))
sns.barplot(x='review_score',y='qtd',data=df_review_score_count)

In [ ]:
# agregação pelo valor pago e frete
df_agg_rs_price_freight = df.groupby('review_score').agg({'review_score':'count','price':'sum','freight_value':'sum'})
df_agg_rs_price_freight.columns = ['rs_count','price_sum','freight_value_sum']
df_agg_rs_price_freight = df_agg_rs_price_freight.reset_index()

# fez-se então os dois percentuais juntos
df_agg_rs_price_freight['rs_percent'] = (df_agg_rs_price_freight['rs_count']/df_agg_rs_price_freight['rs_count'].sum())*100
df_agg_rs_price_freight['price_sum_percent'] = (df_agg_rs_price_freight['price_sum']/df_agg_rs_price_freight['price_sum'].sum())*100
df_agg_rs_price_freight['freight_value_sum_percent'] = (df_agg_rs_price_freight['freight_value_sum']/df_agg_rs_price_freight['freight_value_sum'].sum())*100

display(df_agg_rs_price_freight)

In [ ]:
# agrupamento dos elementos de review_score

df['review_score'] = df['review_score'].apply(lambda x: 1 if x > 3 else 0)

print(df['review_score'])

In [ ]:
# fazendo um describe de todos os campos

display(df.describe())

In [ ]:
var_num = []
var_cat = []

for col in list(df.columns):
    if df[col].dtype != 'object':
        var_num.append(col)
    else:
        var_cat.append(col)
        
print(var_num)
print(var_cat)

In [ ]:
var_num_plot = ['customer_city_idhm', 'seller_city_idhm', 'product_photos_qty', 'price', 'freight_value', 'dias_entrega']

nrows = len(var_num_plot)*2
ncols = 1
index_num = 0

# fig = plt.figure(figsize=(20,60))
# fig.subplots_adjust(wspace=0.01,hspace=0.4)

for col in var_num_plot:
        
        fig = plt.figure(figsize=(20,60))
        
        index_num = index_num + 1
        if index_num == 13:
            break
        plt.subplot(nrows,ncols,index_num)
        plt.title('Histograma de {}'.format(col))
        plt.xlabel(col)
        sns.histplot(data=df[col])

        plt.grid()

        index_num = index_num + 1
        if index_num == 13:
            break
        plt.subplot(nrows, ncols, index_num)
        plt.title('Boxplot de {}'.format(col))
        plt.xlabel(col)
        sns.boxplot(x=df[col])

        plt.grid()
        
        plt.show()
        
        # cálculo dos outliers
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 -Q1
        upper_cap = Q3 + 1.5*IQR
        lower_cap = Q1 - 1.5*IQR
        print('Q1: ',Q1)
        print('Q3: ',Q3)
        print('IQR: ',IQR)
        print('upper_cap: ',upper_cap)
        print('lower_cap:',lower_cap)
    
    


In [ ]:
# heatmap da matriz de correlação para as variaveis numericas sem o target

df_num = df[var_num_plot]
display(df_num.head())

In [ ]:
fig = plt.figure(figsize=(10,10))

df_num_corr = df_num.corr()

mask_triu = np.triu(df_num_corr,k=0)

plt.title('Matriz de Correlação das Variáveis Nméricas')
sns.heatmap(df_num_corr, annot=True, fmt='.2f', square=True, cmap = 'Blues_r', mask=mask_triu)



In [ ]:
# fig = plt.figure(figsize=(7,7))
# sns.pairplot(data=df,hue='review_score')
# plt.show()

## Separação do dataframe de treinamento da variável de resultado

In [ ]:
# variavel resultado

y = df['review_score'].copy()

print(y)

In [ ]:
# variaveis preditoras

X = df.drop(columns='review_score').copy()

display(X)

In [ ]:
display(X.columns)

## Codificação das variáveis categóricas para numéricas

In [ ]:
# LableEncoder para codificar as variaveis categoricas como numeros

encoder = preprocessing.LabelEncoder()

In [ ]:
for col in list(X.columns):
    if X[col].dtype == 'object':
        X[col] = encoder.fit_transform(X[col])

In [ ]:
display(X)

## Resultados dos Modelos

### Função que treina o modelo e mostra as metricas dos resultados

In [ ]:
# cálculo e resultado do modelo

def calc_and_metrics(X,y,modelo):
    
    # separação da base de teste (20% do total de linhas) e da base de teste (80% do total de linhas)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state = 11)
    
    # treinamento do modelo
    classificador = modelo(random_state=11)
    classificador = classificador.fit(X_train,y_train)
    y_pred = classificador.predict(X_test)

    # confusion matrix

    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)

    print('\n')

    print(confusion_matrix(y_test,y_pred))

    print('\n')

    soma_pred_true = tp+fp+fn+tn

    print('soma tp,fp,fn,tp: ', soma_pred_true)
    print('len y_test: ',len(y_test))

    plot_confusion_matrix(classificador,X_test,y_test, labels=[1,0], cmap='Blues')
    plt.show()

    # metricas do resultado do modelo

    precision = np.round(tp/(tp+fp),2)
    recall = np.round(tp/(tp+fn),2)
    accuracy = np.round((tp+tn)/soma_pred_true,2)
    f_score = np.round((2*recall*precision)/(recall+precision),2)
    print('precision: ',precision*100)
    print('recall: ',recall*100)
    print('F-score: ',f_score)
    print('accuracy test set: ',accuracy*100)

    print('\n')

    # acuracia da base de teste e da base de treino

    train_accuracy = np.round(classificador.score(X_train, y_train), 2)
    test_accuracy = np.round(classificador.score(X_test, y_test), 2)

    print("Training Set Mean Accuracy = " + str(train_accuracy))
    print("Test Set Mean Accuracy = " + str(test_accuracy))

    # importancia das variaveis no modelo

    df_ft_importance = pd.DataFrame({'feature_names':X.columns,'importance':classificador.feature_importances_})
    display(df_ft_importance.sort_values(by='importance',ascending=False))
    
    # ROC and AUC
    
    plot_roc_curve(classificador,X_test,y_test)
    plt.show()

### Funão específica da árvore de decisão para a poda

In [ ]:
def arvore_decisao_poda(X,y,max_depth):
    
    # separação da base de teste (20% do total de linhas) e da base de teste (80% do total de linhas)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state = 11)
    
    # treinamento arvore de decisao
    classificador = DecisionTreeClassifier(max_depth=max_depth,random_state=11)
    classificador = classificador.fit(X_train,y_train)
    y_pred = classificador.predict(X_test)

    # confusion matrix

    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
    print('tn:',tn)
    print('fp:',fp)
    print('fn:',fn)
    print('tp:',tp)

    print('\n')

    print(confusion_matrix(y_test,y_pred))

    print('\n')

    soma_pred_true = tp+fp+fn+tn

    print('soma tp,fp,fn,tp: ', soma_pred_true)
    print('len y_test: ',len(y_test))

    plot_confusion_matrix(classificador,X_test,y_test, labels=[1,0], cmap='Blues')
    plt.show()

    # metricas do resultado do modelo

    precision = np.round(tp/(tp+fp),2)
    recall = np.round(tp/(tp+fn),2)
    accuracy = np.round((tp+tn)/soma_pred_true,2)
    f_score = np.round((2*recall*precision)/(recall+precision),2)
    print('precision: ',precision*100)
    print('recall: ',recall*100)
    print('F-score: ',f_score)
    print('accuracy test set: ',accuracy*100)

    print('\n')

    # acuracia da base de teste e da base de treino

    train_accuracy = np.round(classificador.score(X_train, y_train), 2)
    test_accuracy = np.round(classificador.score(X_test, y_test), 2)

    print("Training Set Mean Accuracy = " + str(train_accuracy))
    print("Test Set Mean Accuracy = " + str(test_accuracy))

    # importancia das variaveis no modelo

    df_ft_importance = pd.DataFrame({'feature_names':X.columns,'importance':classificador.feature_importances_})
    display(df_ft_importance.sort_values(by='importance',ascending=False))
    
    plot_roc_curve(classificador,X_test,y_test)
    plt.show()

### Decicion Tree

In [ ]:
calc_and_metrics(X,y,DecisionTreeClassifier)

### Random Forest

In [ ]:
calc_and_metrics(X,y,RandomForestClassifier)

## Avaliação do Desbalanceamento da variável resposta

In [ ]:
ax = y.value_counts().plot(kind='bar', figsize=(10, 6), fontsize=13, color='#087E8B')
ax.set_title('0 = rejeitado, 1 = aprovado)', size=20, pad=30)
ax.set_ylabel('Number of transactions', fontsize=14)

print('percentual da diferença: ',str(round(((np.sum(y==1)-np.sum(y==0))/len(y))*100)),'%')

for i in ax.patches:
    ax.text(i.get_x(), i.get_height() + 700, str(round(i.get_height(), 2)) + '  ' + '(' +str(round((i.get_height()/len(y))*100, 2)) + ' %' + ')', fontsize=15)
    


### Aplicando o SMOTE nas variáveis

In [ ]:
# aplicando o SMOTE

sm = SMOTE(random_state=10)

X_sm, y_sm = sm.fit_resample(X,y)

ax = y_sm.value_counts().plot(kind='bar', figsize=(10, 6), fontsize=13, color='#087E8B')
ax.set_title('0 = rejeitado, 1 = aprovado)', size=20, pad=30)
ax.set_ylabel('Number of transactions', fontsize=14)

for i in ax.patches:
    ax.text(i.get_x() + 0.19, i.get_height() + 700, str(round(i.get_height(), 2)), fontsize=15)

## Treinamento e resultados numericos para o dataframe transformado com o SMOTE

In [ ]:
# classificador,X_test,y_test = calc_and_metrics(X_sm,y_sm,DecisionTreeClassifier)
calc_and_metrics(X_sm,y_sm,DecisionTreeClassifier)

In [ ]:
# Alteração nos hyperparametros para ver se o modelo tem resultado melhor

In [ ]:
# Random Forest
calc_and_metrics(X_sm,y_sm,RandomForestClassifier)

## Podando a arvore de decisao

In [ ]:
# separação da base de teste (20% do total de linhas) e da base de teste (80% do total de linhas)
X_train_sm,X_test_sm,y_train_sm,y_test_sm = train_test_split(X_sm,y_sm,test_size=0.2, random_state = 11)

# alterando para 10 a quantidade de ramos desde a raiz
clf_dt = DecisionTreeClassifier(random_state=11)
clf_dt = clf_dt.fit(X_train_sm,y_train_sm)

dt_depth = clf_dt.get_depth()

print(clf_dt.get_params())
print('\n')
print('qtd ramos desde raiz: ', dt_depth)
print('\n')
print('qtd folhas arvore: ',clf_dt.get_n_leaves())

In [ ]:
# resultados as alterações do hyperparametro de 1 até dt_depth

lista_resultado = []

for depth in range(1,dt_depth + 1):
                      
        classifier = DecisionTreeClassifier(max_depth=depth,random_state=11)
        classifier = classifier.fit(X_train_sm,y_train_sm)
        y_pred_sm = classifier.predict(X_test_sm)

        # confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_test_sm,y_pred_sm).ravel()
        
        precision = np.round(tp/(tp+fp),2)
        recall = np.round(tp/(tp+fn),2)
        f_score = np.round((2*recall*precision)/(recall+precision),2)
        
        train_accuracy = np.round(classifier.score(X_train_sm, y_train_sm), 2)
        test_accuracy = np.round(classifier.score(X_test_sm, y_test_sm), 2)
        
        lista_resultado.append([depth,precision,recall,f_score,train_accuracy,test_accuracy])
        
# print(lista_resultado)

In [ ]:
df_resultados_pruning = pd.DataFrame(lista_resultado,columns=['max_depth','precision','recall','f_score','train_accuracy','test_accuracy'])
display(df_resultados_pruning.sort_values(by=['test_accuracy','f_score','precision','recall'],ascending=[False,False,False,False]).head())

In [ ]:
arvore_decisao_poda(X_sm,y_sm,39)